# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [4]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lae,PG,-6.7221,146.9847,75.87,63,100,6.13,overcast clouds
1,1,Rikitea,PF,-23.1203,-134.9692,77.90,70,69,3.15,broken clouds
2,2,Dingle,PH,10.9995,122.6711,74.07,95,100,7.18,overcast clouds
3,3,Sorong,ID,-0.8833,131.2500,80.64,80,97,9.71,overcast clouds
4,4,Pevek,RU,69.7008,170.3133,-31.20,100,100,4.54,overcast clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [6]:
# Prompt the user to enter minimum and maximum temperature criteria.
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip? '))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [7]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & \
                                      (city_data_df['Max Temp'] >= min_temp)]

# Display sample data
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lae,PG,-6.7221,146.9847,75.87,63,100,6.13,overcast clouds
1,1,Rikitea,PF,-23.1203,-134.9692,77.90,70,69,3.15,broken clouds
2,2,Dingle,PH,10.9995,122.6711,74.07,95,100,7.18,overcast clouds
3,3,Sorong,ID,-0.8833,131.2500,80.64,80,97,9.71,overcast clouds
5,5,Saint-Louis,SN,16.3333,-15.0000,80.24,12,99,10.09,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
690,690,Voh,NC,-20.9667,164.7000,79.79,73,33,6.60,scattered clouds
691,691,Cape Canaveral,US,28.4058,-80.6048,74.05,71,0,5.75,clear sky
694,694,Rio Gallegos,AR,-51.6226,-69.2181,71.02,39,2,18.05,clear sky
695,695,Catamarca,AR,-28.4696,-65.7852,82.58,46,76,10.71,broken clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [8]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities_df = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lae,PG,-6.7221,146.9847,75.87,63,100,6.13,overcast clouds
1,1,Rikitea,PF,-23.1203,-134.9692,77.90,70,69,3.15,broken clouds
2,2,Dingle,PH,10.9995,122.6711,74.07,95,100,7.18,overcast clouds
3,3,Sorong,ID,-0.8833,131.2500,80.64,80,97,9.71,overcast clouds
5,5,Saint-Louis,SN,16.3333,-15.0000,80.24,12,99,10.09,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
690,690,Voh,NC,-20.9667,164.7000,79.79,73,33,6.60,scattered clouds
691,691,Cape Canaveral,US,28.4058,-80.6048,74.05,71,0,5.75,clear sky
694,694,Rio Gallegos,AR,-51.6226,-69.2181,71.02,39,2,18.05,clear sky
695,695,Catamarca,AR,-28.4696,-65.7852,82.58,46,76,10.71,broken clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [9]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [10]:
# Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Lae,PG,75.87,overcast clouds,-6.7221,146.9847,
1,Rikitea,PF,77.90,broken clouds,-23.1203,-134.9692,
2,Dingle,PH,74.07,overcast clouds,10.9995,122.6711,
3,Sorong,ID,80.64,overcast clouds,-0.8833,131.2500,
5,Saint-Louis,SN,80.24,overcast clouds,16.3333,-15.0000,
...,...,...,...,...,...,...,...
690,Voh,NC,79.79,scattered clouds,-20.9667,164.7000,
691,Cape Canaveral,US,74.05,clear sky,28.4058,-80.6048,
694,Rio Gallegos,AR,71.02,clear sky,-51.6226,-69.2181,
695,Catamarca,AR,82.58,broken clouds,-28.4696,-65.7852,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [11]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [13]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Lae - nearest hotel: Huon Gulf Hotel
Rikitea - nearest hotel: Chez Bianca & Benoit
Dingle - nearest hotel: No hotel found
Sorong - nearest hotel: Hotel Membramo Marina
Saint-Louis - nearest hotel: No hotel found
Hilo - nearest hotel: Dolphin Bay Hotel
Mar Del Plata - nearest hotel: Nuevo Ostende
Bonthe - nearest hotel: No hotel found
Rawson - nearest hotel: Hotel Deportivo
Butaritari - nearest hotel: No hotel found
Lethem - nearest hotel: Hotel Amazonas
Sekoma - nearest hotel: No hotel found
Belawan - nearest hotel: Danau Toba International Cottage Belawan
Lorengau - nearest hotel: Seeadler Bay Hotel
Chuy - nearest hotel: Alerces
Avarua - nearest hotel: Paradise Inn
San Cristobal - nearest hotel: Terra Sur
Puerto Ayora - nearest hotel: Hostal La Mirada De Solitario George
Kapaa - nearest hotel: Pono Kai Resort
Hermanus - nearest hotel: Aloe guest house
Kavieng - nearest hotel: Kavieng Hotel
Mahebourg - nearest hotel: Grand Bel Air
Bengkulu - nearest hotel: Dena Ho

Morondava - nearest hotel: Hotel Sharon Menabe
Hobyo - nearest hotel: No hotel found
Ancud - nearest hotel: Cabañas
Mathathane - nearest hotel: No hotel found
Oranjestad - nearest hotel: Hotel California
Alyangula - nearest hotel: No hotel found
Taoudenni - nearest hotel: No hotel found
Lavumisa - nearest hotel: No hotel found
Sri Aman - nearest hotel: Mega Inn
Siyabuswa - nearest hotel: No hotel found
Kaili - nearest hotel: No hotel found
Brasileia - nearest hotel: Pousada Azeveda
Toma - nearest hotel: No hotel found
Gamba - nearest hotel: No hotel found
Ilo - nearest hotel: Hostal Central
Touros - nearest hotel: Pousada Atlântico
Antalaha - nearest hotel: Hôtel Florida
Port Blair - nearest hotel: Andaman Ocean Breeze
Mitsamiouli - nearest hotel: Hotel le Paradis
San Carlos - nearest hotel: Hotel Villas del Norte
Delvada - nearest hotel: No hotel found
Monrovia - nearest hotel: Bella Casa Hotel
Maceio - nearest hotel: Maceió Hostel e Pousada
Bairiki - nearest hotel: Marry's Motel
Tual

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Lae,PG,75.87,overcast clouds,-6.7221,146.9847,Huon Gulf Hotel
1,Rikitea,PF,77.90,broken clouds,-23.1203,-134.9692,Chez Bianca & Benoit
2,Dingle,PH,74.07,overcast clouds,10.9995,122.6711,No hotel found
3,Sorong,ID,80.64,overcast clouds,-0.8833,131.2500,Hotel Membramo Marina
5,Saint-Louis,SN,80.24,overcast clouds,16.3333,-15.0000,No hotel found
...,...,...,...,...,...,...,...
690,Voh,NC,79.79,scattered clouds,-20.9667,164.7000,Le Vook
691,Cape Canaveral,US,74.05,clear sky,28.4058,-80.6048,Holiday Inn Club Vacations Cape Canaveral Beac...
694,Rio Gallegos,AR,71.02,clear sky,-51.6226,-69.2181,Amérian Río Gallegos Apart & Suites
695,Catamarca,AR,82.58,broken clouds,-28.4696,-65.7852,Hotel de Turismo


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [27]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.drop(hotel_df[(hotel_df['Hotel Name'] == 'No hotel found')].index)

# Display sample data
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Lae,PG,75.87,overcast clouds,-6.7221,146.9847,Huon Gulf Hotel
1,Rikitea,PF,77.90,broken clouds,-23.1203,-134.9692,Chez Bianca & Benoit
3,Sorong,ID,80.64,overcast clouds,-0.8833,131.2500,Hotel Membramo Marina
7,Hilo,US,82.13,scattered clouds,19.7297,-155.0900,Dolphin Bay Hotel
10,Mar Del Plata,AR,75.22,moderate rain,-38.0023,-57.5575,Nuevo Ostende
16,Rawson,AR,78.46,broken clouds,-43.3002,-65.1023,Hotel Deportivo
20,Lethem,GY,89.65,scattered clouds,3.3803,-59.7968,Hotel Amazonas
23,Belawan,ID,75.49,broken clouds,3.7755,98.6832,Danau Toba International Cottage Belawan
24,Lorengau,PG,81.63,overcast clouds,-2.0226,147.2712,Seeadler Bay Hotel
26,Chuy,UY,73.58,scattered clouds,-33.6971,-53.4616,Alerces


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [29]:
# Create the output File (CSV)
output_file = 'WeatherPy_vacation.csv'

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_file, index='City_ID')

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [30]:
# Configure the map plot
city_plot = clean_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ['City','Hotel Name','Country','Description'],
    alpha= 0.7
)

# Display the map
city_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country)

AttributeError: module 'matplotlib.pyplot' has no attribute 'get_figure'